쿠팡 다이나믹의 4/3~5/9 구매 데이터를 활용한 분석 코드. 재구매, 네이버서치api 활용한 집계 등을 수행함

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
import openpyxl
pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [137]:
df = pd.read_csv('../result/result_0403_0509.csv',encoding='utf-8-sig')

In [138]:
df = df[df['subParam'].notna()]

In [139]:
df['subParam'] = df['subParam'].apply(lambda x: x.split('_')[0])

In [43]:
grouped = df.groupby(['subParam','productName'])['orderDate'].apply(list).reset_index()
repeated_purchases = grouped[grouped['orderDate'].apply(lambda x: len(set(x)) > 1)]

In [127]:
df.subParam.nunique()

26905

In [44]:
len(repeated_purchases) / len(df)

0.007637145002200534

In [52]:
repeated_purchases.subParam.nunique()

448

In [53]:
repeated_purchases.subParam.nunique() / df.subParam.nunique()

0.016651180078052406

In [46]:
df.orderDate.min(),df.orderDate.max()

(20240403, 20240509)

In [58]:
repeated_purchases['len'] = repeated_purchases['orderDate'].apply(lambda x : len(x))

repeated_purchases.to_csv('../재구매.csv')

C:\Users\Owner\AppData\Local\Temp\ipykernel_17296\2751100586.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repeated_purchases['len'] = repeated_purchases['orderDate'].apply(lambda x : len(x))


동일상품 재구매/총 재구매

In [63]:
param = df.groupby('subParam')['orderDate'].nunique().reset_index()
len(param[param['orderDate']>1])

6485

In [69]:
repeated_purchases.subParam.nunique() / param[param['orderDate']>1].subParam.nunique()

0.06908249807247495

In [73]:
1266/df.orderId.nunique(),df.orderId.nunique()

(0.02742396672731999, 46164)

In [89]:
info = pd.read_csv("../../cp_data.csv")
client_id = info.iloc[2,1]
client_secret = info.iloc[2,2]

In [91]:
#-*- coding: utf-8 -*-
import os
import sys
import urllib.request
client_id = client_id
client_secret = client_secret
url = "https://openapi.naver.com/v1/datalab/shopping/categories"
body = "{\"startDate\":\"2017-08-01\",\"endDate\":\"2017-09-30\",\"timeUnit\":\"month\",\"category\":[{\"name\":\"패션의류\",\"param\":[\"50000000\"]},{\"name\":\"화장품/미용\",\"param\":[\"50000002\"]}],\"device\":\"pc\",\"ages\":[\"20\",\"30\"],\"gender\":\"f\"}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"startDate":"2017-08-01","endDate":"2017-09-30","timeUnit":"month","results":[{"title":"패션의류","category":["50000000"],"data":[{"period":"2017-08-01","ratio":84.01252},{"period":"2017-09-01","ratio":100}]},{"title":"화장품/미용","category":["50000002"],"data":[{"period":"2017-08-01","ratio":22.21162},{"period":"2017-09-01","ratio":21.54278}]}]}


In [100]:
product_names = repeated_purchases.productName.tolist()

In [104]:
import os
import sys
import urllib.request
import json
import pandas as pd

info = pd.read_csv("../../cp_data.csv")
client_id = info.iloc[3,1]
client_secret = info.iloc[3,2]

def get_category(product_name):
    # 네이버 쇼핑 검색 API URL
    url = f"https://openapi.naver.com/v1/search/shop.json?query={urllib.parse.quote(product_name)}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if rescode == 200:
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))
        if data['items']:
            # 검색 결과의 첫 번째 상품의 카테고리 정보 추출
            item = data['items'][0]
            category1 = item.get('category1', '')
            category2 = item.get('category2', '')
            category3 = item.get('category3', '')
            category4 = item.get('category4', '')
            return [product_name, category1, category2, category3, category4]
        else:
            return [product_name, '', '', '', '']
    else:
        print("Error Code:" + rescode)
        return [product_name, '', '', '', '']

# 예제 상품명 리스트

# 결과 저장 리스트
results = []

# 각 상품명에 대해 카테고리 정보 가져오기
for product_name in product_names:
    result = get_category(product_name)
    results.append(result)

# DataFrame으로 변환
df = pd.DataFrame(results, columns=['Product Name', 'Category1', 'Category2', 'Category3', 'Category4'])

# 결과 출력
print(df)


                                          Product Name Category1   Category2  \
0                     MIZ DAY 여아 주니어 초경량 고탄력 일체형 브라 3P     출산/육아  유아동언더웨어/잠옷   
1                        프라임큐 삼성 갤럭시 호환 C타입 유선 오픈형 이어폰    디지털/가전        음향가전   
2                            [로켓프레시] 슬림쿡 쉬림프 크림파스타 샐러드                         
3                       도씨 여성용 실크터치 스포츠 심리스 브라 캡 나시 런닝                         
4                              허닭 한입 닭가슴살 큐브 떡갈비맛 (냉동)        식품         축산물   
5                            닥터스베스트 글루코사민 콘드로이틴 MSM 캡슐        식품        건강식품   
6                       닥터스베스트 엑스트라 스트렝스 징코 120mg 베지 캡        식품        건강식품   
7                                            벨레다 소금 치약     생활/건강      구강위생용품   
8                                돌체구스토 디카페인 아메리카노 캡슐커피        식품          음료   
9                                              몽베스트 생수        식품          음료   
10                                 프렌치카페 로스터리 돌체 연유 라떼        식품          음료   
11                     동원 추억의 부산어묵 부드러운 

In [111]:
len(repeated_purchases)

590

In [115]:
# df.rename(columns={'Product Name':'productName'},inplace=True)
mgd = pd.merge(repeated_purchases,prd,on='productName',how='left')
len(mgd)

590

In [112]:
prd = df.copy()
prd.drop_duplicates(inplace=True)
len(df) - len(prd)


85

In [116]:
prd.to_csv('../정리된 데이터.csv')

In [120]:
mgd.to_csv('../정리된 데이터.csv')

In [123]:
prd.groupby('Category1')['Category2'].unique()

Category1
                                                          []
가구/인테리어       [침실가구, 인테리어소품, 거실가구, 서재/사무용가구, 수예, 수납가구, 침구단품]
디지털/가전     [음향가전, 노트북액세서리, 계절가전, 휴대폰액세서리, 영상가전, 주방가전, 주변기...
생활/건강      [구강위생용품, 자동차용품, 반려동물, 생활용품, 공구, 주방용품, 건강관리용품, ...
스포츠/레저                        [요가/필라테스, 헬스, 축구, 스포츠액세서리, 수영]
식품         [축산물, 건강식품, 음료, 냉동/간편조리식품, 농산물, 과자/베이커리, 반찬, 유...
여가/생활편의                                               [생활편의]
출산/육아      [유아동언더웨어/잠옷, 분유, 유아동잡화, 소독/살균용품, 유아동의류, 아기간식, ...
패션의류                      [여성언더웨어/잠옷, 여성의류, 남성의류, 남성언더웨어/잠옷]
패션잡화              [남성신발, 여행용가방/소품, 패션소품, 헤어액세서리, 여성신발, 신발용품]
화장품/미용     [선케어, 헤어케어, 스킨케어, 헤어스타일링, 색조메이크업, 바디케어, 클렌징, 뷰...
Name: Category2, dtype: object

In [128]:
df.head()

,Unnamed: 0,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission,date,refund_quantity,refund_gmv,refund_commission,total_gmv,total_commission,total_quantity
0,0,20240403,AF3155932,homecoin,d5ac3b54-1ba4-42c0-be8f-28a3ce3f4eb5,460,327924416,8244947215083538,7215473660,미농찰진 2023년 찹쌀,1,11980.0,5.25,629.0,0,0.0,0.0,0.0,11980.0,629.0,1.0
5,5,20240403,AF3155932,homecoin,cfc66f94-cc08-4124-a29a-83f3e8155362,460,7613011741,8364941415097516,5829298628,맥 텐더토크 립 밤,1,29870.0,5.25,1569.0,0,0.0,0.0,0.0,29870.0,1569.0,1.0
6,6,20240403,AF3155932,homecoin,d9dbcf8e-2f43-4002-a3e2-ce052526c909,460,6113203139,8054946615792965,115749717,마또네 후레쉬모짜슬라이스치즈 (냉동),1,16960.0,5.25,891.0,0,0.0,0.0,0.0,16960.0,891.0,1.0
10,10,20240403,AF3155932,homecoin,89576b6a-9cbc-4f40-aa52-259a73913c1a,460,7706141860,8864943615832129,257233459,푸드팜 피칸 햇 캘리포니아 500g,1,27087.0,5.25,1423.0,0,0.0,0.0,0.0,27087.0,1423.0,1.0
12,12,20240403,AF3155932,homecoin,401acff0-da76-404b-b98f-ccdf87e7e9ea,400,7774803171,8874947815886312,7290972662,참크래커,4,3440.0,5.25,181.0,0,0.0,0.0,0.0,3440.0,181.0,4.0


In [143]:
# 각 유저별로 구매 날짜를 정렬합니다.
df = df.sort_values(by=['subParam', 'orderDate'])
df['orderDate'] = pd.to_datetime(df['orderDate'], format='%Y%m%d')
# 재구매까지 걸린 시간 계산
def calculate_days_between_purchases(dates):
    sorted_dates = sorted(dates)
    return [(sorted_dates[i+1] - sorted_dates[i]).days for i in range(len(sorted_dates) - 1)]

# 각 유저별로 최초 구매와 재구매 사이의 일수를 계산합니다.
days_between_purchases = df.groupby('subParam')['orderDate'].apply(calculate_days_between_purchases)

# 재구매까지 걸린 모든 일수를 하나의 리스트로 합칩니다.
all_days = [day for sublist in days_between_purchases for day in sublist]

# 재구매까지 걸린 시간의 평균 계산
average_days = sum(all_days) / len(all_days) if all_days else float('nan')

print(f"재구매까지 걸린 시간의 평균: {average_days}일")

재구매까지 걸린 시간의 평균: 1.4874774076942938일


In [146]:
df['orderDate'] = pd.to_datetime(df['orderDate'], format='%Y%m%d')

# 각 유저별로 구매 날짜를 정렬합니다.
df = df.sort_values(by=['subParam', 'orderDate'])

# 2회 이상 구매한 유저들만 필터링
purchase_counts = df['subParam'].value_counts()
repeat_buyers = purchase_counts[purchase_counts > 1].index
df_repeat_buyers = df[df['subParam'].isin(repeat_buyers)]

# 재구매까지 걸린 시간 계산
def calculate_days_between_purchases(dates):
    sorted_dates = sorted(dates)
    return [(sorted_dates[i+1] - sorted_dates[i]).days for i in range(len(sorted_dates) - 1)]

# 각 유저별로 최초 구매와 재구매 사이의 일수를 계산합니다.
days_between_purchases = df_repeat_buyers.groupby('subParam')['orderDate'].apply(calculate_days_between_purchases)

# 재구매까지 걸린 모든 일수를 하나의 리스트로 합칩니다.
all_days = [day for sublist in days_between_purchases for day in sublist]

# 재구매까지 걸린 시간의 평균 계산
average_days = sum(all_days) / len(all_days) if all_days else float('nan')

print(f"재구매까지 걸린 시간의 평균: {average_days}일")

재구매까지 걸린 시간의 평균: 1.4874774076942938일


In [148]:
len(repeat_buyers)

14804

In [149]:
# 각 유저별로 구매 날짜를 정렬합니다.
df = df.sort_values(by=['subParam', 'orderDate'])

# 2회 이상 구매한 유저들만 필터링
purchase_counts = df['subParam'].value_counts()
repeat_buyers = purchase_counts[purchase_counts > 1].index
df_repeat_buyers = df[df['subParam'].isin(repeat_buyers)]

# 같은 날에 여러 번 구매한 경우를 한 번의 주문으로 간주하여 고유한 날짜만 남깁니다.
df_repeat_buyers_unique_dates = df_repeat_buyers.drop_duplicates(subset=['subParam', 'orderDate'])

# 재구매까지 걸린 시간 계산
def calculate_days_between_purchases(dates):
    sorted_dates = sorted(dates)
    return [(sorted_dates[i+1] - sorted_dates[i]).days for i in range(len(sorted_dates) - 1)]

# 각 유저별로 최초 구매와 재구매 사이의 일수를 계산합니다.
days_between_purchases = df_repeat_buyers_unique_dates.groupby('subParam')['orderDate'].apply(calculate_days_between_purchases)

# 재구매까지 걸린 모든 일수를 하나의 리스트로 합칩니다.
all_days = [day for sublist in days_between_purchases for day in sublist]

# 재구매까지 걸린 시간의 평균 계산
average_days = sum(all_days) / len(all_days) if all_days else float('nan')

print(f"재구매까지 걸린 시간의 평균: {average_days}일")

재구매까지 걸린 시간의 평균: 7.330951448707909일


In [153]:
param = df.groupby('subParam')['orderDate'].nunique().reset_index()
len(param[param['orderDate']>1])

6485

In [161]:
rpt_pcs = list(set(param[param['orderDate']>1].subParam.tolist()))
rpt_df = df[df['subParam'].isin(rpt_pcs)]

In [162]:
rpt_df_unique_dates = rpt_df.drop_duplicates(subset=['subParam', 'orderDate'])

# 재구매까지 걸린 시간 계산
def calculate_days_between_purchases(dates):
    sorted_dates = sorted(dates)
    return [(sorted_dates[i+1] - sorted_dates[i]).days for i in range(len(sorted_dates) - 1)]

# 각 유저별로 최초 구매와 재구매 사이의 일수를 계산합니다.
days_between_purchases = rpt_df_unique_dates.groupby('subParam')['orderDate'].apply(calculate_days_between_purchases)

# 재구매까지 걸린 모든 일수를 하나의 리스트로 합칩니다.
all_days = [day for sublist in days_between_purchases for day in sublist]

# 재구매까지 걸린 시간의 평균 계산
average_days = sum(all_days) / len(all_days) if all_days else float('nan')

print(f"재구매까지 걸린 시간의 평균: {average_days}일")

재구매까지 걸린 시간의 평균: 7.330951448707909일


In [166]:
rpt_df = df[df['subParam'].isin(mgd.subParam.tolist())]
rpt_df_unique_dates = rpt_df.drop_duplicates(subset=['subParam', 'orderDate'])

# 재구매까지 걸린 시간 계산
def calculate_days_between_purchases(dates):
    sorted_dates = sorted(dates)
    return [(sorted_dates[i+1] - sorted_dates[i]).days for i in range(len(sorted_dates) - 1)]

# 각 유저별로 최초 구매와 재구매 사이의 일수를 계산합니다.
days_between_purchases = rpt_df_unique_dates.groupby('subParam')['orderDate'].apply(calculate_days_between_purchases)

# 재구매까지 걸린 모든 일수를 하나의 리스트로 합칩니다.
all_days = [day for sublist in days_between_purchases for day in sublist]

# 재구매까지 걸린 시간의 평균 계산
average_days = sum(all_days) / len(all_days) if all_days else float('nan')

print(f"재구매까지 걸린 시간의 평균: {average_days}일")

재구매까지 걸린 시간의 평균: 5.478334749362786일


In [170]:
len(mgd[mgd['Category1']=='식품'].subParam)

354

In [176]:
rpt_df = df[df['subParam'].isin(mgd[mgd['Category1']=='디지털/가전'].subParam.tolist())]
rpt_df_unique_dates = rpt_df.drop_duplicates(subset=['subParam', 'orderDate'])

# 재구매까지 걸린 시간 계산
def calculate_days_between_purchases(dates):
    sorted_dates = sorted(dates)
    return [(sorted_dates[i+1] - sorted_dates[i]).days for i in range(len(sorted_dates) - 1)]

# 각 유저별로 최초 구매와 재구매 사이의 일수를 계산합니다.
days_between_purchases = rpt_df_unique_dates.groupby('subParam')['orderDate'].apply(calculate_days_between_purchases)

# 재구매까지 걸린 모든 일수를 하나의 리스트로 합칩니다.
all_days = [day for sublist in days_between_purchases for day in sublist]

# 재구매까지 걸린 시간의 평균 계산
average_days = sum(all_days) / len(all_days) if all_days else float('nan')

print(f"재구매까지 걸린 시간의 평균: {average_days}일")

재구매까지 걸린 시간의 평균: 4.0일
